## Jupyter Notebook: Qualitätsoptimierung mit Causal AI

### Problemstellung
#### Ziel ist, die Qualität der Produktion durch die gezielte Einstellung von Maschinenparametern mit Hilfe von Causal AI zu verbessern.

In [1]:
# Schritt 1: Vorbereitung der Daten

# Beispiel: Einlesen der Daten
import pandas as pd

data = pd.read_csv('maschinen_daten.csv')

# Datenübersicht
print(data.head())

      Drehzahl  Temperatur  Spannung  Materialqualität  Qualität
0  1374.540119  228.871320  2.666196          0.759541  0.656171
1  1950.714306  217.526746  5.877109          0.739591  0.865405
2  1731.993942  201.666933  8.856513          0.512821  0.562166
3  1598.658484  236.965651  7.590024          0.670624  0.651972
4  1156.018640  227.931182  8.259050          0.690098  0.678418


In [2]:
# Schritt 2: Kausales Modell definieren (mit DoWhy)
from dowhy import CausalModel

causal_graph = """
digraph {
    Drehzahl -> Temperatur;
    Temperatur -> Spannung;
    Spannung -> Qualität;
    Materialqualität -> Qualität;
}"""

model = CausalModel(data=data,
                    treatment='Temperatur',
                    outcome='Qualität',
                    graph=causal_graph)

identified_estimand = model.identify_effect()
print(identified_estimand)

causal_estimate = model.estimate_effect(identified_estimand,
                                        method_name="backdoor.linear_regression")
print("Kausaler Effekt der Temperatur auf Qualität:", causal_estimate.value)

C:\Users\michi\anaconda3\envs\master-thesis-prototyp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                   
─────────────(E[Qualität])
d[Temperatur]             
Estimand assumption 1, Unconfoundedness: If U→{Temperatur} and U→Qualität then P(Qualität|Temperatur,,U) = P(Qualität|Temperatur,)

### Estimand : 2
Estimand name: iv
Estimand expression:
 ⎡                                                 -1⎤
 ⎢     d                ⎛     d                   ⎞  ⎥
E⎢───────────(Qualität)⋅⎜───────────([Temperatur])⎟  ⎥
 ⎣d[Drehzahl]           ⎝d[Drehzahl]              ⎠  ⎦
Estimand assumption 1, As-if-random: If U→→Qualität then ¬(U →→{Drehzahl})
Estimand assumption 2, Exclusion: If we remove {Drehzahl}→{Temperatur}, then ¬({Drehzahl}→Qualität)

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
 ⎡     d                      d                  ⎤
E⎢───────────(Qualität)⋅─────────────([Spannung])⎥
 ⎣d[Spannung]           d[Temperatur]            ⎦
Estimand a

C:\Users\michi\anaconda3\envs\master-thesis-prototyp\Lib\site-packages\dowhy\causal_estimator.py:266: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  by_effect_mods = data.groupby(effect_modifier_names)
C:\Users\michi\anaconda3\envs\master-thesis-prototyp\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
C:\Users\michi\anaconda3\envs\master-thesis-prototyp\Lib\site-packages\dowhy\causal_estimators\regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a f

In [3]:
# Schritt 3: Ursachenanalyse bei schlechter Qualität
schlechte_qualitaet = data[data['Qualität'] < 0.5]
ursachen = schlechte_qualitaet.mean().sort_values()
print("Ursachen schlechter Qualität:")
print(ursachen)

Ursachen schlechter Qualität:
Qualität               0.432288
Materialqualität       0.711034
Spannung               6.391867
Temperatur           200.752233
Drehzahl            1358.032161
dtype: float64


In [4]:
# Schritt 4: Counterfactual-Analyse (EconML)
from econml.dml import LinearDML

X = data[['Drehzahl', 'Spannung', 'Materialqualität']]
T = data['Temperatur']
Y = data['Qualität']

estimator = LinearDML()
estimator.fit(Y, T, X=X)

counterfactual_effect = estimator.effect(X, T0=220, T1=215).mean()
print("Effekt einer Temperaturreduktion um 5 Grad:", counterfactual_effect)

Effekt einer Temperaturreduktion um 5 Grad: -0.011468694372512037


In [5]:
# Schritt 5: Modell zur Qualitätsvorhersage erstellen
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(data[['Drehzahl', 'Temperatur', 'Spannung', 'Materialqualität']], data['Qualität'])

RandomForestRegressor()

In [7]:
# Beispiel korrigierter Code
test_param = pd.DataFrame([[1500, 220, 5, 0.9]], 
                          columns=['Drehzahl', 'Temperatur', 'Spannung', 'Materialqualität'])

predicted_quality = rf_model.predict(test_param)
print("Vorhergesagte Qualität:", predicted_quality[0])

Vorhergesagte Qualität: 0.8267732172680069
